In [6]:
import zlib
from random import shuffle

def ram_shuffle(filename_in, filename_out, header=True):
    with open(filename_in, 'rb') as f:
        zlines = [zlib.compress(line,9) for line in f]
        if header:
            first_row = zlines.pop(0)
    shuffle(zlines)
    
    with open(filename_out, 'wb') as f:
        if header:
            f.write(zlib.decompress(first_row))
        for zline in zlines:
            f.write(zlib.decompress(zline))

In [7]:
import os
local_path = os.getcwd()
source = 'datasets/covtype.data'
ram_shuffle(filename_in = local_path+'/'+source, \
            filename_out = local_path+'/datasets/shuffled_covtype.data', \
            header = False)


In [52]:
import csv, time
import numpy as np
from sklearn.linear_model import SGDClassifier
source = 'datasets/shuffled_covtype.data'
SEP=','
forest_type = [t+1 for t in range(7)]
SGD = SGDClassifier(loss='log', penalty=None, random_state=1, average=True)
accuracy = 0
holdout_count = 0
prog_accuracy = 0
prog_count = 0
cold_start = 200000
k_holdout = 10

with open(local_path+'/'+source, 'r') as R:
    iterator = csv.reader(R, delimiter=SEP)
    for n, row in enumerate(iterator):
        if n>250000:
            break
            
        # data processing
        response = np.array([int(row[-1])])
        features = np.array(list(map(float,row[:-1]))).reshape(1,-1)
                            
        #machine learning
        if n+1 >= cold_start and (n+1-cold_start) % k_holdout==0: # validate after cold_start and validate one of 10
            if int(SGD.predict(features)) == response[0]:
                accuracy += 1
            holdout_count += 1
            if (n+1-cold_start) % 25000 == 0 and (n+1) > cold_start:
                print ('%s holdout accuracy: %0.3f' % (time.strftime('%X'), accuracy/float(holdout_count)))
        else:
            # progressive validation
            if (n+1) >= cold_start:
                if int(SGD.predict(features)) == response[0]:
                    prog_accuracy += 1
                prog_count += 1
                if n % 25000 == 0 and n > cold_start:
                    print ('%s progressive accuracy: %0.3f' % (time.strftime('%X'), prog_accuracy/float(prog_count)))
            
            # learning phase
            SGD.partial_fit(features, response, classes=forest_type)
    
    print ('%s FINAL holdout accuracy: %0.3f' % (time.strftime('%X'), accuracy/((n+1-cold_start)/float(k_holdout))))
    print ('%s FINAL progressive accuracy: %0.3f' % (time.strftime('%X'), prog_accuracy/float(prog_count)))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


01:52:13 holdout accuracy: 0.628
01:52:13 progressive accuracy: 0.616
01:53:16 holdout accuracy: 0.629
01:53:16 progressive accuracy: 0.617
01:53:16 FINAL holdout accuracy: 0.630
01:53:16 FINAL progressive accuracy: 0.617


In [46]:
arr = np.array(list(map(float, [1,2,3,4,5]))).reshape(-1,1)
arr.dtype
n=10
if n > 5:
    print('true')

true
